In [1]:
import requests
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import osm2geojson  # pip install osm2geojson

# Bounding box for Lyon
north, south = 45.827515314683616, 45.63329334299378
west, east = 4.720509943289811, 5.145971094584033

# Overpass query: both ways and relations for buildings
overpass_url = "https://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:180];
(
  way["building"]({south},{west},{north},{east});
  relation["building"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

# Request the data
print("⏳ Downloading OSM data for Lyon...")
response = requests.get(overpass_url, params={"data": query})
data = response.json()

# Convert OSM JSON to GeoJSON
print("🔄 Converting to GeoJSON...")
geojson = osm2geojson.json2geojson(data)

# Load into GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson["features"])
gdf.set_crs("EPSG:4326", inplace=True)

# Fix any invalid geometries
print("🧼 Cleaning geometries...")
gdf["geometry"] = gdf["geometry"].buffer(0)

# Merge all building geometries
print("🔗 Merging all buildings...")
merged = unary_union(gdf["geometry"])

# Wrap in GeoDataFrame
if isinstance(merged, MultiPolygon):
    merged_gdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=gdf.crs)
else:
    merged_gdf = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)

# Save to file
output_file = "lyon_buildings_merged.gpkg"
merged_gdf.to_file(output_file, driver="GPKG")

print(f"✅ Done! Merged {len(gdf)} buildings into {len(merged_gdf)} shapes.")
print(f"📦 Saved to: {output_file}")


⏳ Downloading OSM data for Lyon...
🔄 Converting to GeoJSON...
🧼 Cleaning geometries...
🔗 Merging all buildings...


/Users/gabeiras3j/opt/anaconda3/envs/cleanproj/lib/python3.10/site-packages/shapely/set_operations.py:543: RuntimeWarning: invalid value encountered in create_collection
  collections = lib.create_collection(


✅ Done! Merged 345334 buildings into 180053 shapes.
📦 Saved to: lyon_buildings_merged.gpkg
